In [27]:
import os
import requests
import fitz
from tqdm.auto import tqdm
import pandas as pd
from spacy.lang.en import English
import random
import re

In [4]:
# pdf_path = "" #Change to any document of choice
pdf_path = "human_nutrition_text.pdf"

# if not os.path.exists(pdf_path):
#   print("File to be downloaded")
#   url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
#   filename = pdf_path
#   response = requests.get(url)
#   if response.status_code == 200:
#     with open(filename, "wb") as file:
#       file.write(response.content)
#     print(f"File {filename} has been downloaded and saved")
#   else:
#     print(f"File could not be downloaded. Status code: {response.status_code}")
# else:
#   print(f"File {pdf_path} already exists")
  

In [14]:
def text_formatter(text: str) -> str:
  cleaned_text = text.replace("\n", " ").strip()
  return cleaned_text

def open_and_read_pdf(pdf: str) -> list[dict]:
    doc = fitz.open(pdf)
    pages_and_text = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_text.append({"page_number": page_number-4,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count": len(text.split(". ")),
                                "page_token_count": len(text)/4, # 1 token = 4 characters (assumption)
                                "text": text}) 
    return pages_and_text

pages = open_and_read_pdf(pdf = pdf_path)
pages[4:6]

0it [00:00, ?it/s]

[{'page_number': 0,
  'page_char_count': 450,
  'page_word_count': 78,
  'page_sentence_count': 5,
  'page_token_count': 112.5,
  'text': '5 New BRS Navigation    Path: KITAAB homepage > Record-Keeping > Bank Reconciliation Statement  Background :  Every transaction in the Ledger is compared with the transactions in the Bank Statement to confirm if they match or not and vice-versa. Objective:  The objective of a bank reconciliation statement is to Identify the 1. Payments and receipts that are not recorded in Ledger. 2. Payments and receipts that are not found in the Bank Statement.'},
 {'page_number': 1,
  'page_char_count': 1099,
  'page_word_count': 220,
  'page_sentence_count': 12,
  'page_token_count': 274.75,
  'text': '6 Problem Statement for BRS Problem Statement:  If a BRS is absent, it can lead to significant financial inconsistencies. Handling this process manually is  time-consuming and raises the likelihood of unnoticed errors occurring.   Prerequisites :    We are dividin

In [16]:
df = pd.DataFrame(pages)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,text
0,-4,2183,893,8,545.75,New BRS 5 Problem Statement for BR...
1,-3,2447,1163,9,611.75,Calculation of Gross Profit & Net Profit 72 ...
2,-2,2149,1117,7,537.25,Functionality- Screen & Fields. 152 ...
3,-1,639,314,1,159.75,Statutory - GST 224 Stat...
4,0,450,78,5,112.50,5 New BRS Navigation Path: KITAAB homepage ...


In [17]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count
count,252.00,252.00,252.00,252.00,252.00
mean,121.50,926.71,182.56,9.07,231.68
std,72.89,682.70,158.96,9.59,170.67
min,-4.00,2.00,1.00,1.00,0.50
25%,58.75,358.75,64.50,3.00,89.69
50%,121.50,864.50,158.00,6.50,216.12
75%,184.25,1443.75,272.25,12.00,360.94
max,247.00,3253.00,1163.00,61.00,813.25


In [19]:
nlp = English()
# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/ 
nlp.add_pipe("sentencizer")
# Create a document instance as an example
doc = nlp("This is a sentence. This another sentence.")
assert len(list(doc.sents)) == 2
# Access the sentences of the document
list(doc.sents)

[This is a sentence., This another sentence.]

In [20]:
for item in tqdm(pages):
    item["sentences"] = list(nlp(item["text"]).sents)
    
    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    # Count the sentences 
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/252 [00:00<?, ?it/s]

In [25]:
random.sample(pages, k=1)

[{'page_number': 198,
  'page_char_count': 1058,
  'page_word_count': 197,
  'page_sentence_count': 18,
  'page_token_count': 264.5,
  'text': "203 i. Check all the Ledgers for  TDS_Applicability_Type (DB Field), if  it is ‘Expense’ then treat it as Expense  Ledger and consider it only if it is  ‘Credited'. ii. Exception: Do not consider the  Ledgers that are created under the  following groups; 1. Primary - Incomes 2. Direct Incomes 3. Indirect Incomes 4. Sales c. Incase this voucher is NOT linked with  “Booking Voucher”, then it will be  considered here. Case 3: Reversal of Deduction Voucher a. TDS Ledger - It should be Debited in the  Purchase/Journal Voucher. TDS Ledgers  means the ledger created under the  group “Duties & Taxes” and Type of Duty  is “TDS”. b. Party Ledger - Check all the Ledgers for  TDS_Applicability_Type (DB Field), if it is  ‘Party’ then treat it as Party Ledger and  consider it only if it is ‘Credited'. c. Incase this voucher is NOT linked with  “Deduction Vou

In [26]:
df = pd.DataFrame(pages)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,page_sentence_count_spacy
count,252.00,252.00,252.00,252.00,252.00,252.00
mean,121.50,926.71,182.56,9.07,231.68,8.32
std,72.89,682.70,158.96,9.59,170.67,8.01
min,-4.00,2.00,1.00,1.00,0.50,1.00
25%,58.75,358.75,64.50,3.00,89.69,3.00
50%,121.50,864.50,158.00,6.50,216.12,6.00
75%,184.25,1443.75,272.25,12.00,360.94,11.00
max,247.00,3253.00,1163.00,61.00,813.25,53.00


In [29]:
num_sentence_chunk_size = 5

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/252 [00:00<?, ?it/s]

In [31]:
# Sample an example from the group (note: many samples have only 1 chunk as they have <=10 sentences total)
random.sample(pages, k=1)

[{'page_number': 144,
  'page_char_count': 1213,
  'page_word_count': 379,
  'page_sentence_count': 11,
  'page_token_count': 303.25,
  'text': '149 Example -Data Table with Bill wise view Bill Allocation Table (Raw Data Table) Follow the below steps on ‘Raw Table’ to get the Data table with ‘Bill wise’ pending balance.   1. Calculation of ‘Net Amount’ for every unique Bill ID.  The ‘Net Amount’ will calculate “Add all Dr and Less all Cr” .  Net Amount= All ‘Dr (-) All Cr 2. As a result, every unique Bill ID will have one entry.  3. Now filter the ‘Bill ID’, where Amount type is in ‘Cr’.  4. This filtered table is the base data for Bill Payable Report with Bill wise.  For Reference,  Bills Payable Report (Bill wise view) a1  1          20  Dr  a1  2          30  Cr  a3  4          30  Cr  a5  5          40  Dr  a1  1          50  Dr  Ledger  name  Bill ID  Voucher  ID  Reference  Type  (Adv, New  Ref, Adv Ref,  On account)  Due Date  Credit  Days  Amount  Amount  Type  a1  1          7

In [32]:
pages_and_chunks = []
for item in tqdm(pages):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
        
        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

  0%|          | 0/252 [00:00<?, ?it/s]

535

In [33]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 14,
  'sentence_chunk': 'Fetch list of all transactions of the ledger statement for the ledger_id 2. Filter voucher_type as “payment” (all voucher_types under Payment) 3. Filter the transactions that have the reconciliation status “missing” 4. Calculate the total of all transactions and display the”total” here These screens are explained in detail below. Receipts missing in books This is the clickable section that redirects to the Payments missing in the Books detailed screen.',
  'chunk_char_count': 456,
  'chunk_word_count': 69,
  'chunk_token_count': 114.0}]

In [34]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,535.0,535.00,535.00,535.00
mean,134.3,423.57,73.59,105.89
std,73.8,298.09,57.60,74.52
min,-4.0,2.00,1.00,0.50
25%,63.5,222.50,38.00,55.62
50%,151.0,381.00,63.00,95.25
75%,195.0,553.50,94.50,138.38
max,247.0,1923.00,503.00,480.75


In [35]:
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 0.75 | Text: 111
Chunk token count: 12.75 | Text: Field Name  Type Screen Functionality / Validations
Chunk token count: 5.75 | Text: 236 SI - Standard Rates
Chunk token count: 0.5 | Text: 69
Chunk token count: 22.25 | Text: 2. Deductee Type is not provided in TDS Analysis.3. TCS is not enabled in Party Ledger.1.


In [41]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -4,
  'sentence_chunk': 'New BRS 5       Problem Statement for BRS 6       1. List of Ledgers 7         Components for List of Ledgers 8       2. Bank Reconciliation Statement 10         Components For BRS 11       3. Bank Statement 13       4. Ledger Statement 15       Logic for Status 17       5.',
  'chunk_char_count': 274,
  'chunk_word_count': 99,
  'chunk_token_count': 68.5},
 {'page_number': -4,
  'sentence_chunk': 'Reconciliation Statement: Summary 18         Calculation of Differences 21         Reconciliation Statement: Matched 23         Reconciliation statement: Missing in Books 24         Reconciliation Statement: Missing in Bank 25       6. Payments Missing in Bank 26         6.1 Mapping 27         6.2 Mapping Data Table 29       7. Other Payments and Recipts screen 31       Upload Bank Statement in BRS 33         Auto-Matching column names: 36    Create Voucher Type 37       Side Panel - Voucher Numbering 40       Details - Tab 41       Series - Tab 42  

In [42]:
from sentence_transformers import SentenceTransformer

In [43]:
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device="cpu") # choose the device to load the model to (note: GPU will often be *much* faster than CPU)

# Create a list of sentences to turn into numbers
sentences = [
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]

# Sentences are encoded/embedded by calling model.encode()
embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

# See the embeddings
for sentence, embedding in embeddings_dict.items():
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\debgh\AllProjects\SimpleRAGImplementation\.venv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\debgh\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Sentence: The Sentences Transformers library provides an easy and open-source way to create embeddings.
Embedding: [-2.07983200e-02  3.03164814e-02 -2.01217812e-02  6.86484948e-02
 -2.55256109e-02 -8.47687945e-03 -2.07220146e-04 -6.32377192e-02
  2.81606615e-02 -3.33353654e-02  3.02633923e-02  5.30721210e-02
 -5.03526852e-02  2.62288619e-02  3.33313867e-02 -4.51577455e-02
  3.63044776e-02 -1.37122895e-03 -1.20171504e-02  1.14947017e-02
  5.04511073e-02  4.70856912e-02  2.11913809e-02  5.14606461e-02
 -2.03746762e-02 -3.58889364e-02 -6.67759334e-04 -2.94393804e-02
  4.95859124e-02 -1.05639528e-02 -1.52014270e-02 -1.31761038e-03
  4.48197536e-02  1.56023586e-02  8.60379373e-07 -1.21391879e-03
 -2.37978902e-02 -9.09328402e-04  7.34484056e-03 -2.53934250e-03
  5.23370542e-02 -4.68043610e-02  1.66214500e-02  4.71579805e-02
 -4.15599346e-02  9.01977066e-04  3.60278189e-02  3.42214108e-02
  9.68227237e-02  5.94829135e-02 -1.64984465e-02 -3.51249352e-02
  5.92518784e-03 -7.07923085e-04 -2.4103

### CPU Based embedding creation

In [44]:
%%time

# Uncomment to see how long it takes to create embeddings on CPU
# Make sure the model is on the CPU
embedding_model.to("cpu")

# Embed each chunk one by one
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/468 [00:00<?, ?it/s]

CPU times: total: 5min 4s
Wall time: 2min 17s


### For GPU Run Below 

In [ ]:
# %%time

# # Send the model to the GPU
# embedding_model.to("cuda") # requires a GPU installed, for reference on my local machine, I'm using a NVIDIA RTX 4090

# # Create embeddings one by one on the GPU
# for item in tqdm(pages_and_chunks_over_min_token_len):
#     item["embedding"] = embedding_model.encode(item["sentence_chunk"])

In [45]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

### Batch embeddings for shorter times on CPU embeddings

In [50]:
%%time

# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=16, # Change batch size according to speed obtained
                                               convert_to_tensor=True) # optional to return embeddings as tensor instead of array

text_chunk_embeddings

CPU times: total: 7min 11s
Wall time: 2min 57s


tensor([[ 0.0029, -0.0341, -0.0137,  ...,  0.0162, -0.0621, -0.0052],
        [ 0.0207, -0.0004, -0.0371,  ..., -0.0081, -0.0209, -0.0216],
        [-0.0276, -0.0258, -0.0300,  ..., -0.0423,  0.0240, -0.0325],
        ...,
        [ 0.0064, -0.0601, -0.0116,  ...,  0.0031, -0.0104, -0.0028],
        [ 0.0083, -0.0544,  0.0006,  ...,  0.0180, -0.0100,  0.0252],
        [ 0.0023, -0.0445, -0.0415,  ...,  0.0303, -0.0396, -0.0601]])

In [47]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [48]:
# Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-4,New BRS 5 Problem Statement for BRS 6 ...,274,99,68.50,[ 2.93612643e-03 -3.40591148e-02 -1.36541240e-...
1,-4,Reconciliation Statement: Summary 18 C...,1578,464,394.50,[ 2.06729006e-02 -3.63658852e-04 -3.70985456e-...
2,-3,Calculation of Gross Profit & Net Profit 72 ...,987,359,246.75,[-2.76358128e-02 -2.58415826e-02 -2.99574248e-...
3,-3,113 Vertical Layout of Balance Sh...,976,321,244.00,[-2.39460301e-02 -5.23959063e-02 -2.36781035e-...
4,-2,Functionality- Screen & Fields.152 ...,1272,503,318.00,[-2.09647268e-02 -2.22982764e-02 -1.58743616e-...
